## Import Statements

In [1]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna
import keras_tuner as kt
from keras import backend as K

2024-11-28 07:26:58.848147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732778819.279190    2760 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732778819.412017    2760 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 07:27:00.671090: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load Datasets

In [3]:
# define the directory where the pickle files are stored
folder_path = '../data/batched_data_pickle_files/'

# define the filenames for the pickle files
file_names = ['processed_train_data.pkl', 'processed_train_targets.pkl', 'processed_test_data.pkl', 'true_rul.pkl']

# loop through each file and load its contents as arrays
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    
    # read the pickle file
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    
    # ensure data is a numpy array, if not convert it
    if isinstance(data, np.ndarray):
        globals()[file_name.replace('.pkl', '')] = data
    else:
        globals()[file_name.replace('.pkl', '')] = np.array(data)

# print the shapes of the loaded data
print("processed train data shape:", processed_train_data.shape)
print("processed train target shape:", processed_train_targets.shape)
print("processed test data shape:", processed_test_data.shape)
print("true rul shape:", true_rul.shape)

processed train data shape: (17731, 30, 14)
processed train target shape: (17731,)
processed test data shape: (100, 30, 14)
true rul shape: (100,)


## Model Building 

In [12]:
# split the data into training and validation sets
# here, we're taking 20% of the data as the validation set and 80% for training
processed_train_data, processed_val_data, processed_train_targets, processed_val_targets = train_test_split(
    processed_train_data, processed_train_targets, test_size=0.2, random_state=42
)

# define the function to create the lstm model, this will be called by optuna during optimization
def create_model(trial):
    model = Sequential()
    
    # the input shape is derived from the training data
    input_shape = processed_train_data.shape[1:]  # shape excluding the batch size

    # suggest the number of lstm layers (1 to 3 layers)
    num_lstm_layers = trial.suggest_int('num_lstm_layers', 1, 3)
    
    # add the suggested number of lstm layers
    for i in range(num_lstm_layers):
        # for the first lstm layer, define the input shape
        if i == 0:
            model.add(layers.LSTM(
                units=trial.suggest_int(f'lstm_units_{i}', 32, 256),  # lstm units for this layer (trial parameter)
                input_shape=input_shape,  # shape of the input data
                return_sequences=(i < num_lstm_layers - 1),  # if it's not the last lstm layer, return sequences
                activation="tanh"  # using tanh activation function
            ))
        else:
            # for subsequent lstm layers, do not redefine the input shape
            model.add(layers.LSTM(
                units=trial.suggest_int(f'lstm_units_{i}', 32, 256),  # lstm units for this layer (trial parameter)
                return_sequences=(i < num_lstm_layers - 1),  # return sequences for all but the last lstm layer
                activation="tanh"  # using tanh activation function
            ))

    # suggest the number of dense layers (1 to 2 layers)
    num_dense_layers = trial.suggest_int('num_dense_layers', 1, 2)
    
    # add dense layers based on the number suggested
    for j in range(num_dense_layers):
        model.add(layers.Dense(
            units=trial.suggest_int(f'dense_units_{j}', 32, 256),  # number of units in this dense layer
            activation="relu"  # relu activation function for dense layers
        ))

    # add the final output layer with a single neuron (for regression output)
    model.add(layers.Dense(1))

    # suggest a learning rate for the optimizer (log-uniform distribution between 1e-4 and 1e-2)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    
    # compile the model using adam optimizer and mse loss
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss="mse")
    
    return model

# define the objective function that optuna will optimize
def objective(trial):
    # create the model using the current trial's hyperparameters
    model = create_model(trial)
    
    # suggest the batch size for training (32, 64, or 128)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    
    # train the model on the training data with validation data for early stopping
    history = model.fit(
        processed_train_data, processed_train_targets,
        validation_data=(processed_val_data, processed_val_targets),
        epochs=10,  # set the number of epochs for each trial
        batch_size=batch_size,  # batch size defined by optuna
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)],  # early stopping based on validation loss
        verbose=0  # set verbose to 0 to suppress output during training
    )
    
    # predict the values for the validation set
    val_predictions = model.predict(processed_val_data)
    
    # calculate the rmse (root mean squared error) for the validation predictions
    val_rmse = np.sqrt(mean_squared_error(processed_val_targets, val_predictions))
    
    # return the rmse as the objective value (optuna will try to minimize this value)
    return val_rmse

# create an optuna study to minimize the rmse (validation error)
study = optuna.create_study(direction='minimize')

# run the optimization for 50 trials
study.optimize(objective, n_trials=50)

# print the best trial found by optuna
print('best trial:')
trial = study.best_trial
print(f'  value (rmse): {trial.value:.4f}')
print('  params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

# build the best model using the best hyperparameters from the trial
best_model = create_model(trial)

# train the best model on the full training data using the batch size from the best trial
best_model.fit(
    processed_train_data, processed_train_targets,
    epochs=10,  # training the model for 10 epochs
    batch_size=trial.params['batch_size'],  # use the batch size from the best trial
    verbose=0  # suppress output during training
)

# save the best model to a file so it can be used later
best_model.save('best_rul_lstm_model_optuna.h5')
print("best model saved to 'best_rul_lstm_model_optuna.h5'")

# evaluate the best model on the test data
test_predictions = best_model.predict(processed_test_data)

# calculate rmse for the test data predictions
test_rmse = np.sqrt(mean_squared_error(true_rul, test_predictions))

# print the test rmse value
print(f"test rmse: {test_rmse:.4f}")

[I 2024-11-27 06:44:20,950] A new study created in memory with name: no-name-cc1c81e3-b586-4865-8642-59f507b1ff2e
/tmp/ipykernel_2863/4248733727.py:42: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step


[I 2024-11-27 06:46:30,662] Trial 0 finished with value: 14.746357692433431 and parameters: {'num_lstm_layers': 3, 'lstm_units_0': 84, 'lstm_units_1': 175, 'lstm_units_2': 108, 'num_dense_layers': 1, 'dense_units_0': 53, 'learning_rate': 0.00014838057462947094, 'batch_size': 32}. Best is trial 0 with value: 14.746357692433431.
/home/ec2-user/.local/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/tmp/ipykernel_2863/4248733727.py:42: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


[I 2024-11-27 06:46:40,937] Trial 1 finished with value: 17.07468713454603 and parameters: {'num_lstm_layers': 1, 'lstm_units_0': 59, 'num_dense_layers': 2, 'dense_units_0': 42, 'dense_units_1': 176, 'learning_rate': 0.006820342144074775, 'batch_size': 64}. Best is trial 0 with value: 14.746357692433431.
/home/ec2-user/.local/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/tmp/ipykernel_2863/4248733727.py:42: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


...

46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step


[I 2024-11-27 08:32:56,190] Trial 49 finished with value: 11.314191448066346 and parameters: {'num_lstm_layers': 2, 'lstm_units_0': 41, 'lstm_units_1': 232, 'num_dense_layers': 1, 'dense_units_0': 109, 'learning_rate': 0.001555007651239742, 'batch_size': 32}. Best is trial 49 with value: 11.314191448066346.


Best trial:
  Value (RMSE): 11.3142
  Params: 
    num_lstm_layers: 2
    lstm_units_0: 41
    lstm_units_1: 232
    num_dense_layers: 1
    dense_units_0: 109
    learning_rate: 0.001555007651239742
    batch_size: 32


/home/ec2-user/.local/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/tmp/ipykernel_2863/4248733727.py:42: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


Best model saved to 'best_rul_lstm_model_optuna.h5'
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step
Test RMSE: 15.8244


## Evaluation

In [4]:
# load the saved model whilst specifying the loss function
loaded_model = load_model('best_rul_lstm_model_optuna.h5', custom_objects={'mse': MeanSquaredError()})

# make predictions on the test data
test_predictions = loaded_model.predict(processed_test_data)

# calc rmse on the test data
test_rmse = np.sqrt(mean_squared_error(true_rul, test_predictions))

# print rmse
print(f"Test RMSE: {test_rmse:.4f}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step
Test RMSE: 15.8244


In [6]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def evaluate_rul_metrics(true, predicted):
    
    true = np.array(true)
    predicted = np.array(predicted)
    
    mae = float(mean_absolute_error(true, predicted))
    mse = float(mean_squared_error(true, predicted))
    rmse = float(np.sqrt(mse))
    mape = float(np.mean(np.abs((true - predicted) / true)) * 100)
    
    return {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "MAPE": mape
    }
    
metrics = evaluate_rul_metrics(true_rul, test_predictions)
metrics

{'MAE': 12.212742862701417,
 'MSE': 250.41169730769147,
 'RMSE': 15.824401957347124,
 'MAPE': 146.33581180060284}